In [450]:
import numpy as np
import pandas as pd
import requests
import uuid
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def get_uuid():
    return str(uuid.uuid4())

In [359]:
class Spider():
    def __init__(self, name):
        self.name = name
    def spider(self, url = 'https://bj.lianjia.com/ershoufang/pg3rs%E9%9C%8D%E8%90%A5/', proxies_flag = True):
        ua_info = UserAgent()
        cookie_info = 'JSESSIONID=' + get_uuid() + ';' + \
        'user_trace_token=' + get_uuid() + '; LGUID=' + get_uuid() + '; index_location_city=%E6%88%90%E9%83%BD;' + \
        'SEARCH_ID=' + get_uuid() + '; _gid=GA1.2.717841549.1514043316;' +  \
        '_ga=GA1.2.952298646.1514043316;'\
        'LGSID=' + get_uuid() + ';' +  \
        'LGRID=' + get_uuid() + ';'
        headers_info = {'cookie': cookie_info,\
               'user-agent': ua_info.random,\
               'Host':'bj.lianjia.com'}
        proxies_info = {'http':'http://10.240.49.201:8080',\
            'https':'https://10.240.49.201:8080'}
        try:
            s = requests.Session()
            if proxies_flag:
                response = s.get(url, headers=headers_info, proxies = proxies_info)
            else:
                response = s.get(url, headers=headers_info)
            if (response.status_code == 200):
                soup = BeautifulSoup(response.content, 'html.parser')
            else:
                soup = None
            return soup
        except Exception as e:
            print(e)
    def get_house_id(self, pn = 5, url = 'https://bj.lianjia.com/ershoufang/rs%E9%9C%8D%E8%90%A5/'):
        id_list = []
        for loop in range(1, pn+1):
            #print(loop)
            if loop == 1:
                url_info = url
            else:
                url_part1 = re.findall('(.*ershoufang/).*', url)[0]
                url_part2 = 'pg' + str(loop)
                url_part3 = re.findall('.*(rs.*)', url)[0]
                url_info = url_part1 + url_part2 + url_part3
            soup = self.spider(url = url_info)
            id_list_base = str(soup.find_all('div', attrs={'class':'info clear'}))
            reg = re.compile('href.*?/(\d*?).html')
            id_list.extend(re.findall(reg, id_list_base))
        return id_list

In [360]:
lianjia_spider = Spider('lianjia')

In [303]:
#id_list = lianjia_spider.get_house_id(pn=9)

In [104]:
def build_data(id_list):
    data = pd.DataFrame()
    for loop in range(len(id_list)):
        data.loc[loop, 'id'] = id_list[loop]
    return data

In [105]:
data = build_data(id_list)

In [107]:
data.head()

,id
0,101102874856
1,101102828901
2,101102882210
3,101102879591
4,101102874184


In [157]:
data.to_csv('data.csv')

In [361]:
def get_house_info(data, begin_index = 0, end_index = 1):
    try:
        for loop in range(begin_index, end_index):
            print(data.loc[loop, 'id'])
            url_info = 'https://bj.lianjia.com/ershoufang/%s.html' %(data.loc[loop, 'id'])
            soup = lianjia_spider.spider(url = url_info)
            #print(soup)
            if (soup != None):
                data.loc[loop, 'SimpleDescription'] = soup.select('.main')[0].text
                data.loc[loop, 'HousePrice'] = soup.select('.total')[0].text
                data.loc[loop, 'UnitPrice'] = soup.select('.unitPriceValue')[0].text
                data.loc[loop, 'HouseType'] = soup.select('.mainInfo')[0].text
                data.loc[loop, 'HouseDirection'] = soup.select('.mainInfo')[1].text
                data.loc[loop, 'HouseArea'] = soup.select('.mainInfo')[2].text
                data.loc[loop, 'HouseFloor'] = soup.select('.subInfo')[0].text
                data.loc[loop, 'HouseAge'] = soup.select('.subInfo')[2].text
                data.loc[loop, 'CommunityName'] = soup.select('.info')[0].text
                data.loc[loop, 'FollowersCnt'] = soup.select('.count')[0].text
                data.loc[loop, 'WatchersCnt'] = soup.select('.count')[1].text
                data.loc[loop, 'HouseTransType'] = re.findall(r'交易权属</span><span>(.*?)</span>',str(soup.select('.content')).replace('\n', ''))[0]
                data.loc[loop, 'HouseTransAge'] = re.findall(r'房屋年限</span><span>(.*?)</span>',str(soup.select('.content')).replace('\n', ''))[0]
                sleep_random = np.random.randint(5,10)
                time.sleep(sleep_random)
            else:
                pass
        data.to_csv('data.csv', encoding="utf_8_sig", index=False)
    except Exception as e:
        print('Someting wrong here', e)
    return soup

In [301]:
data.shape

(269, 14)

In [362]:
soup = get_house_info(data, begin_index=213, end_index=269)

101102249971
101102243643
101101695315
101102707553
101102689024
101102065420
101102705135
101102749514
101101871075
101101870767
101101872243
101101990397
101101926534
101102652961
101102451129
101102867054
101102823101
101102528910
101102758888
101102170092
101102435399
101102876187
101102249034
101102844981
101101953682
101102655494
101102643673
101102902898
101102889659
101102819953
101102824092
101101088882
101102217867
101102582169
101102855990
101102584560
101101891571
101101891682
101102559002
101102456292
101102785500
101102786900
101102726001
101102789181
101102349914
101102905839
101102902849
101102884413
101102824658
101102826992
101102845723
101102586859
101102219458
101102700260
101102704866
101102794987


In [363]:
data[data.id == '101102249971' ]

,id,SimpleDescription,HousePrice,UnitPrice,HouseType,HouseDirection,HouseArea,HouseFloor,HouseAge,CommunityName,FollowersCnt,WatchersCnt,HouseTransType,HouseTransAge
213,101102249971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [364]:
data.to_csv('data.csv', encoding="utf_8_sig", index=False)

In [366]:
data.tail()

,id,SimpleDescription,HousePrice,UnitPrice,HouseType,HouseDirection,HouseArea,HouseFloor,HouseAge,CommunityName,FollowersCnt,WatchersCnt,HouseTransType,HouseTransAge
264,101102586859,商品房高原值三居室业主换房出售,560,44915元/平米,3室1厅,南 北,124.68平米,高楼层/共6层,2006年建/板楼,龙跃苑东二区,3,0,商品房,满两年
265,101102219458,商品房满两年 南北双通透 看房方便,500,40904元/平米,3室1厅,南 北,122.24平米,顶层/共6层,2006年建/板楼,龙跃苑东四区,25,0,商品房,满两年
266,101102700260,首开国风美唐一期 车位 32万,32,8139元/平米,车位,东,39.32平米,地下室,2012年建,首开国风美唐一期,1,0,商品房,未满两年
267,101102704866,国风美唐B1车位，电梯直接入户,28,8219元/平米,车位,北,34.07平米,地下室,2015年建,首开国风美唐二期,2,0,商品房,未满两年
268,101102794987,车位 诚心出售 位置好 单元楼下,27,7925元/平米,车位,东 南 西 北,34.07平米,地下室,2015年建,首开国风美唐二期,1,0,商品房,未满两年


In [367]:
data.head()

,id,SimpleDescription,HousePrice,UnitPrice,HouseType,HouseDirection,HouseArea,HouseFloor,HouseAge,CommunityName,FollowersCnt,WatchersCnt,HouseTransType,HouseTransAge
0,101102874856,小区中间位置，满五年商品房在京唯一房产,385,51873元/平米,2室1厅,南 北,74.22平米,高楼层/共6层,1995年建/板楼,华龙苑南里,55,51,商品房,满五年
1,101102828901,华龙苑南里一层南北通透两居室，主卧客厅双阳台,377,51126元/平米,2室1厅,南 北,73.74平米,底层/共6层,1995年建/板楼,华龙苑南里,74,46,商品房,满五年
2,101102882210,婚房装修，中间楼层，有地铁8号线和13号线,385,53592元/平米,2室1厅,南,71.84平米,中楼层/共5层,1998年建/板楼,蓝天园,31,5,商品房,满两年
3,101102879591,华龙苑南里三居室+三层+霍营地铁8、13号线,465,44678元/平米,3室1厅,东 南 北,104.08平米,中楼层/共6层,1999年建/板楼,华龙苑南里,17,13,商品房,满五年
4,101102874184,南北通透 客厅卧室各带个阳台 商品房满五年,320,63619元/平米,1室1厅,南 北,50.3平米,中楼层/共6层,1997年建/板楼,华龙苑南里,20,39,商品房,满五年


### Assess
- import the data in 

In [454]:
data = pd.read_csv('data.csv')

In [455]:
data.head()

,id,SimpleDescription,HousePrice,UnitPrice,HouseType,HouseDirection,HouseArea,HouseFloor,HouseAge,CommunityName,FollowersCnt,WatchersCnt,HouseTransType,HouseTransAge
0,101102874856,小区中间位置，满五年商品房在京唯一房产,385.0,51873元/平米,2室1厅,南 北,74.22平米,高楼层/共6层,1995年建/板楼,华龙苑南里,55.0,51.0,商品房,满五年
1,101102828901,华龙苑南里一层南北通透两居室，主卧客厅双阳台,377.0,51126元/平米,2室1厅,南 北,73.74平米,底层/共6层,1995年建/板楼,华龙苑南里,74.0,46.0,商品房,满五年
2,101102882210,婚房装修，中间楼层，有地铁8号线和13号线,385.0,53592元/平米,2室1厅,南,71.84平米,中楼层/共5层,1998年建/板楼,蓝天园,31.0,5.0,商品房,满两年
3,101102879591,华龙苑南里三居室+三层+霍营地铁8、13号线,465.0,44678元/平米,3室1厅,东 南 北,104.08平米,中楼层/共6层,1999年建/板楼,华龙苑南里,17.0,13.0,商品房,满五年
4,101102874184,南北通透 客厅卧室各带个阳台 商品房满五年,320.0,63619元/平米,1室1厅,南 北,50.3平米,中楼层/共6层,1997年建/板楼,华龙苑南里,20.0,39.0,商品房,满五年


In [456]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 14 columns):
id                   269 non-null int64
SimpleDescription    260 non-null object
HousePrice           260 non-null float64
UnitPrice            260 non-null object
HouseType            260 non-null object
HouseDirection       260 non-null object
HouseArea            260 non-null object
HouseFloor           260 non-null object
HouseAge             260 non-null object
CommunityName        260 non-null object
FollowersCnt         260 non-null float64
WatchersCnt          260 non-null float64
HouseTransType       260 non-null object
HouseTransAge        260 non-null object
dtypes: float64(3), int64(1), object(10)
memory usage: 29.5+ KB


In [457]:
data.HouseType.unique()

array(['2室1厅', '3室1厅', '1室1厅', '2室2厅', '3室2厅', nan, '5室2厅', '1室0厅',
       '4室1厅', '4室2厅', '5室1厅', '1房间1卫', '2房间1卫', '3房间1卫', '车位'],
      dtype=object)

In [458]:
len(data[data.HouseType == '车位'])

13

In [459]:
data.HouseAge.unique()

array(['1995年建/板楼', '1998年建/板楼', '1999年建/板楼', '1997年建/板楼', '2000年建/板楼',
       '1992年建/板楼', '1993年建/板楼', '1990年建/板楼', '2003年建/板楼', nan,
       '2009年建/板楼', '2007年建/板楼', '2005年建/塔楼', '2010年建/板塔结合', '2001年建/板楼',
       '2008年建/板楼', '2004年建/板楼', '2005年建/板楼', '2013年建/板楼', '2002年建/板楼',
       '2002年建/板塔结合', '2004年建/板塔结合', '2003年建/塔楼', '2006年建/板楼',
       '2009年建/板塔结合', '2012年建/板楼', '2014年建/板塔结合', '2014年建/板楼',
       '2016年建/板塔结合', '未知年建/板楼', '2014年建/塔楼', '2012年建', '2015年建'],
      dtype=object)

In [460]:
data.HouseFloor.unique()

array(['高楼层/共6层', '底层/共6层', '中楼层/共5层', '中楼层/共6层', '顶层/共6层', '顶层/共5层',
       '低楼层/共6层', '底层/共5层', nan, '低楼层/共9层', '中楼层/共9层', '低楼层/共28层',
       '中楼层/共11层', '中楼层/共10层', '高楼层/共28层', '中楼层/共28层', '底层/共10层',
       '顶层/共9层', '顶层/共8层', '顶层/共10层', '高楼层/共10层', '高楼层/共15层', '低楼层/共18层',
       '中楼层/共32层', '底层/共11层', '中楼层/共18层', '高楼层/共32层', '低楼层/共11层',
       '中楼层/共21层', '高楼层/共11层', '底层/共18层', '高楼层/共18层', '低楼层/共19层',
       '中楼层/共15层', '高楼层/共20层', '顶层/共16层', '底层/共20层', '中楼层/共16层',
       '低楼层/共32层', '低楼层/共16层', '高楼层/共16层', '中楼层/共4层', '中楼层/共17层',
       '顶层/共19层', '低楼层/共17层', '高楼层/共21层', '高楼层/共17层', '底层/共9层', '顶层/共4层',
       '底层/共16层', '地下室'], dtype=object)

In [461]:
data.HouseTransType.unique()

array(['商品房', '私产', '已购公房', nan, '一类经济适用房', '限价商品房'], dtype=object)

In [462]:
data.HouseTransAge.unique()

array(['满五年', '满两年', '未满两年', nan], dtype=object)

In [463]:
data.CommunityName.unique()

array(['华龙苑南里', '蓝天园', '华龙苑中里', nan, '霍营小区', '紫金新干线', '龙锦苑东五区', '龙锦苑东四区',
       '华龙苑北里', '龙锦苑东一区', '龙旗广场', '龙锦苑东二区', '龙锦苑东三区', '天鑫家园', '旺龙花园',
       '紫金新干线二期', '慧龙居', '知本时代', '新康园', '龙跃苑东四区', '龙跃苑东五区', '龙跃苑三区',
       '龙跃苑一区', '旗胜家园', '首开国风美唐一期', '轮胎厂宿舍', '富力桃园C区', '首开国风美唐二期',
       '龙跃苑东二区', '枫丹丽舍', '首开国风美唐三期', '枫丹2号', '首开国风美唐综合楼', '金隅翡丽一期',
       '专家国际花园'], dtype=object)

### Clean
- make the copy data_clean
- drop the missing values rows
- drop the '车位' information
- extract the value for UnitPrice, HouseArea; drop the original columns
- change Community to Category
- save data_clean

In [464]:
data_clean = data.copy()

drop the missing values rows

In [465]:
data_clean = data_clean.dropna()
len(data_clean)

260

drop the '车位' information

In [466]:
chewei_index = data_clean[data_clean.HouseType == '车位'].index
data_clean = data_clean.drop(chewei_index)
len(data_clean)

247

extract the value for UnitPrice, HouseArea

In [467]:
data_clean['UnitPriceV'] = data_clean.UnitPrice.str.extract('(.*)元')
data_clean['HouseAreaV'] = data_clean.HouseArea.str.extract('(.*)平')
data_clean['UnitPriceV'] = data_clean['UnitPriceV'].astype(float)
data_clean['HouseAreaV'] = data_clean['HouseAreaV'].astype(float)
data_clean.head(1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


,id,SimpleDescription,HousePrice,UnitPrice,HouseType,HouseDirection,HouseArea,HouseFloor,HouseAge,CommunityName,FollowersCnt,WatchersCnt,HouseTransType,HouseTransAge,UnitPriceV,HouseAreaV
0,101102874856,小区中间位置，满五年商品房在京唯一房产,385.0,51873元/平米,2室1厅,南 北,74.22平米,高楼层/共6层,1995年建/板楼,华龙苑南里,55.0,51.0,商品房,满五年,51873.0,74.22


In [468]:
data_clean = data_clean.drop(['HouseArea', 'UnitPrice'], axis=1)
data_clean.head()

,id,SimpleDescription,HousePrice,HouseType,HouseDirection,HouseFloor,HouseAge,CommunityName,FollowersCnt,WatchersCnt,HouseTransType,HouseTransAge,UnitPriceV,HouseAreaV
0,101102874856,小区中间位置，满五年商品房在京唯一房产,385.0,2室1厅,南 北,高楼层/共6层,1995年建/板楼,华龙苑南里,55.0,51.0,商品房,满五年,51873.0,74.22
1,101102828901,华龙苑南里一层南北通透两居室，主卧客厅双阳台,377.0,2室1厅,南 北,底层/共6层,1995年建/板楼,华龙苑南里,74.0,46.0,商品房,满五年,51126.0,73.74
2,101102882210,婚房装修，中间楼层，有地铁8号线和13号线,385.0,2室1厅,南,中楼层/共5层,1998年建/板楼,蓝天园,31.0,5.0,商品房,满两年,53592.0,71.84
3,101102879591,华龙苑南里三居室+三层+霍营地铁8、13号线,465.0,3室1厅,东 南 北,中楼层/共6层,1999年建/板楼,华龙苑南里,17.0,13.0,商品房,满五年,44678.0,104.08
4,101102874184,南北通透 客厅卧室各带个阳台 商品房满五年,320.0,1室1厅,南 北,中楼层/共6层,1997年建/板楼,华龙苑南里,20.0,39.0,商品房,满五年,63619.0,50.30


change Community to Category

In [469]:
data_clean['CommunityName'] = data_clean['CommunityName'].astype('category')

In [470]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 0 to 265
Data columns (total 14 columns):
id                   247 non-null int64
SimpleDescription    247 non-null object
HousePrice           247 non-null float64
HouseType            247 non-null object
HouseDirection       247 non-null object
HouseFloor           247 non-null object
HouseAge             247 non-null object
CommunityName        247 non-null category
FollowersCnt         247 non-null float64
WatchersCnt          247 non-null float64
HouseTransType       247 non-null object
HouseTransAge        247 non-null object
UnitPriceV           247 non-null float64
HouseAreaV           247 non-null float64
dtypes: category(1), float64(5), int64(1), object(7)
memory usage: 28.8+ KB


save 

In [471]:
data_clean.to_csv('data_clean.csv', encoding="utf_8_sig", index=False)

### Explore